In [19]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

## Get all univs

In [20]:
univ_table_url = "https://www.4icu.org/fr/"
driver = webdriver.Chrome('C:\\Users\\ghenima\\Downloads\\chromedriver_win32_111\\chromedriver.exe') 
driver.get(univ_table_url)
html = driver.page_source

#### get the univ link list

In [21]:
univ_links_list = []
univ_table = driver.find_elements(By.CSS_SELECTOR, ".table tbody tr td:nth-child(2) a")
for univ in univ_table:
    univ_links_list.append(univ.get_attribute('href'))
#univ_links_list

#### delete the last element from the list

In [22]:
univ_links_list = univ_links_list[:-1]

In [23]:
print(len(univ_links_list))

232


## Function to get info of the univs and to make it as a dict

In [24]:
def get_info_univ(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "lxml")
    #dealing with table
    tabs = soup.find_all("table", class_="table borderless")
    tab_info = {}
    for ind, tab in enumerate(tabs):
        trs = tab.find_all("tr")
        for row in trs:
            content_key = row.find("th").get_text()
            content_value = row.find("td").get_text()
            tab_info[content_key] = content_value
    
    #Dealing with rank column
    rank_table = soup.find_all("table", class_="text-right")
    for ind, tab in enumerate(rank_table):
        trs = [tab.find_all("td")]
        for row in trs:
            tab_info["country rank"] = row[1].get_text()
            tab_info["world rank"] = row[3].get_text()
            
    #dealing with social
    social_box = soup.select("#social-media > div.panel-body")
    content_key_list = ["Facebook", "Twitter", "Linkedin", "Youtube", "Instagram"]
    content_value_list = []
    for index, row in enumerate(social_box):
        if row.find("h4"):
                for r in row.find_all("p"):
                    if r.text == ' n.a.':
                        content_value_list.append("NaN")
                    elif len(r.find_all("a")) == 2:
                        content_value = r.find_all("a")[1]["href"]
                        content_value_list.append(content_value)
                    else:
                        content_value = r.find_all("a")[0]["href"]
                        content_value_list.append(content_value)

    for k, kro in enumerate(content_key_list):
        tab_info[content_key_list[k]] = content_value_list[k]
    
    #Tuition
    tuition = soup.select("body > div:nth-child(3) > div:nth-child(6) > div.panel-body > div > table")
    tuition_key_list = ["undergratuate_local", "under_local", "post_local",
                 "postgraduate_international", "under_inter", "post_inter"]
    tuition_value_list = [j.text for i in tuition for j in i.find_all("td")]
    for t, trow in enumerate(tuition_key_list):
        tab_info[tuition_key_list[t]]=tuition_value_list[t]
    return tab_info

### create univ_info_list 

In [26]:
univ_info_list = []
for l, link in enumerate(univ_links_list):
    #print(l, "|", link)
    univ_info_list.append(get_info_univ(link))

## functions to save data

In [27]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

## Save data

#### Save data as json

In [28]:
save_data("C:\\Users\\ghenima\\Documents\\ghenimac\\UnivData\\FranceUnivData.json", univ_info_list)

#### From json to excel or CSV

In [29]:
df_json = pd.read_json("C:\\Users\\ghenima\\Documents\\ghenimac\\UnivData\\FranceUnivData.json")
df_json.to_excel("C:\\Users\\ghenima\\Documents\\ghenimac\\UnivData\\FranceUnivData.xlsx")

In [30]:
df_json.to_csv("C:\\Users\\ghenima\\Documents\\ghenimac\\UnivData\\FranceUnivData.csv")

#### Create a df from xlsx file

In [31]:
df = pd.read_excel("C:\\Users\\ghenima\\Documents\\ghenimac\\UnivData\\FranceUnivData.xlsx")

In [32]:
display(df)

,Unnamed: 0,Name,Acronym,Founded,Screenshot,Address,Tel,Fax,Other locations,Gender,...,Youtube,Instagram,undergratuate_local,under_local,post_local,postgraduate_international,under_inter,post_inter,Colours,Motto
0,0,University of Strasbourg,NaN,1621,\n\n,4 Rue Blaise Pascal - CS 90032 Strasbourg 670...,+33 (0)3 68 85 00 00,+33 (0)3 68 85 07 33,Colmar,Men and Women (coed),...,https://www.youtube.com/channel/UCdrq17AhGO_yW...,NaN,Local students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",International students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",NaN,NaN
1,1,University of Paris,NaN,2019 Merger between Université Paris 5 René De...,\n\n,85 boulevard Saint-Germain Paris 75006 Ile-de...,+33,+33,NaN,Men and Women (coed),...,https://www.youtube.com/channel/UC5WhlXe1I_eRC...,NaN,Local students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",International students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",NaN,NaN
2,2,University of Lille,NaN,2018 Origins date back to 1562,\n\n,42 rue Paul Duez Lille 59000 Hauts-de-France ...,+33 (0)3 62 26 90 00,+33 (0)3 20 88 24 32,NaN,Men and Women (coed),...,https://www.youtube.com/channel/UCwQRR1SjMMfwd...,NaN,Local students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",International students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",NaN,NaN
3,3,Sorbonne University,SU,2018 Merger between Université Paris-Sorbonne ...,\n\n,21 rue de lÉcole de médecine Paris 75006 Ile...,+33 (0)1 44 27 44 27,+33,Abu Dhabi,Men and Women (coed),...,https://www.youtube.com/channel/UCJRfmVMO9aeYD...,NaN,Local students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",International students,"0-1,000 US$ (0-750 Euro)","0-1,000 US$ (0-750 Euro)",Indigo and yellow,NaN
4,4,Normal Superior School,ENS,1794,\n\n,45 Rue d'Ulm Paris 75230 Ile-de-France France,+33 (0)1 44 32 30 00,+33 (0)1 44 32 20 99,NaN,Men and Women (coed),...,https://www.youtube.com/channel/UCbn8O8WwMeoZs...,NaN,Local students,"0-1,000 US$ (0-750 Euro)",Not reported,International students,"0-1,000 US$ (0-750 Euro)",Not reported,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,227,Adventist University of France,NaN,1921,\n\n,"33, Chemin du Perouzet Collonges-sous-Saleve ...",+33 (0)4 50 87 68 00,+33 (0)4 50 43 65 80,NaN,Men and Women (coed),...,https://www.youtube.com/channel/UCYL1xYzW6actM...,NaN,Local students,"10,000-12,500 US$ (7,400-9,200 Euro)","10,000-12,500 US$ (7,400-9,200 Euro)",International students,"10,000-12,500 US$ (7,400-9,200 Euro)","10,000-12,500 US$ (7,400-9,200 Euro)",NaN,Apprendre pour Servir Learn to Serve
228,228,Graduate School of Building Engineering of Metz,ESITC Metz,1992,\n\n,"6, Rue Marconi Metz 57070 Grand Est France",+33 (0)3 87 76 44 50,+33 (0)3 87 76 16 19,NaN,Men and Women (coed),...,NaN,NaN,Local students,Not reported,Not reported,International students,Not reported,Not reported,NaN,NaN
229,229,Occitanie School of Management and Business,EGC Occitanie,,\n\n,"5, rue de Bruxelles, Bourran Rodez 12000 Occi...",+33 (0)5 65 75 56 86,+33,"Montauban, Tarbes, Nîmes",Men and Women (coed),...,NaN,NaN,Local students,"5,000-7,500 US$ (3,700-5,500 Euro)",Not reported,International students,"5,000-7,500 US$ (3,700-5,500 Euro)",Not reported,NaN,NaN
230,230,Higher Institute of Buildings and Public Works,ISBA-TP,1952,\n\n,5 Rue Enrico Fermi Marseille 13453 Provence-A...,+33 (0)4 91 10 68 65,+33 (0)4 91 10 68 64,NaN,Men and Women (coed),...,NaN,NaN,Local students,Not reported,"5,000-7,500 US$ (3,700-5,500 Euro)",International students,Not reported,"5,000-7,500 US$ (3,700-5,500 Euro)",NaN,NaN
